In [14]:
# Authors: Isabella Silva Ferreira and Micael Veríssimo de Araújo
# e-mails: (is_bella21@hotmail.com)    (micaelvero@hotmail.com)


In [2]:
# Read data File and sapare the all the sets
import os
import pickle
import numpy as np
import time

from sklearn.decomposition import PCA
from sklearn.externals import joblib

print "All Right!"

All Right!


In [77]:
# Confirm our array shape
data.shape

(191100, 13)

In [4]:
# Train NN
#from Functions import LogFunctions as log
from sklearn import cross_validation
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
import keras.callbacks as callbacks
from keras.utils import np_utils

from datetime import datetime  
from datetime import timedelta 

print "All Right!"

Using Theano backend.


ImportError: No module named h5py